# LangChainでラクラク！「PDF Chat」
[公式ドキュメント](https://python.langchain.com/en/latest/modules/chains/index_examples/chat_vector_db.html)  
[参考サイト](https://qiita.com/hiroki_okuhata_int/items/7102bab7d96eb2574e7d)  
[参考動画](https://www.youtube.com/watch?v=eCtHVmXcXMs)  

# LangChainとは

言語モデルを利用してアプリケーションを開発するためのフレームワークです。  
[公式サイト](https://python.langchain.com/en/latest/index.html)  
[日本語参考サイト](https://zenn.dev/umi_mori/books/prompt-engineer/viewer/langchain_overview)  

In [1]:
# ! pip install langchain chromadb pypdf

In [2]:
from pathlib import Path

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader

In [3]:
# API_KEY読み込み
with open(f'{Path.home()}/.private_info/api_key.txt') as f:
    api_key = f.read()

In [4]:
# チャット設定
class PDFChatAI:

    def __init__(self, api_key: str, pdf_path: str, **model_kwargs):
        print('Embedding PDF...')
        # Embeddingモデルの設定
        embeddings = OpenAIEmbeddings(openai_api_key=api_key)
        # PDFをページごとにテキストとして読み込み
        pages = PyPDFLoader(pdf_path).load_and_split()
        # テキストをベクトル化しデータベースに格納
        self.database = Chroma.from_documents(pages, embeddings=embeddings)
        print('Embedding complete!')
        # ChatGPTモデルの設定
        self.llm = ChatOpenAI(model_name='gpt-3.5-turbo', openai_api_key=api_key, **model_kwargs)
        # データベースに対してチャットする設定
        self.chat = ConversationalRetrievalChain.from_llm(llm=self.llm,
                                                          retriever=self.database.as_retriever(),
                                                          chain_type='stuff')
        self.chat_history = []

    def generate(self, question: str):
        # Q&A
        result = self.chat({'question': question, 'chat_history': self.chat_history})
        # チャット履歴を格納
        self.chat_history.append((question, result['answer']))
        return result['answer']

    def clear(self):
        # チャット履歴をクリア
        self.chat_history = []

In [5]:
qa = PDFChatAI(api_key=api_key, pdf_path='./data/transformer.pdf', temperature=0)

Embedding PDF...


Using embedded DuckDB without persistence: data will be transient
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


Embedding complete!


In [6]:
qa.generate('モデルのアーキテクチャについて要点をまとめてください')

'Transformerモデルは、エンコーダとデコーダの2つの部分から構成されます。エンコーダは、入力シーケンスを受け取り、それを連続的な表現に変換します。デコーダは、エンコーダが生成した表現を受け取り、出力シーケンスを生成します。両方の部分には、スタックされたセルフアテンションとポイントワイズ、完全に接続されたレイヤーがあります。エンコーダとデコーダの両方に、位置エンコーディングが追加されます。位置エンコーディングは、入力埋め込みに加えられ、シーケンス内のトークンの位置を示します。位置エンコーディングには、サインとコサイン関数を使用することができます。また、マルチヘッドアテンションと呼ばれる機能を使用して、複数のアテンションレイヤーを並列に実行することができます。'

In [7]:
qa.generate('猫の柄についてまとめてください')

"I'm sorry, but there is no information provided about the pattern of cats in the given context."

In [8]:
qa.chat_history

[('モデルのアーキテクチャについて要点をまとめてください',
  'Transformerモデルは、エンコーダとデコーダの2つの部分から構成されます。エンコーダは、入力シーケンスを受け取り、それを連続的な表現に変換します。デコーダは、エンコーダが生成した表現を受け取り、出力シーケンスを生成します。両方の部分には、スタックされたセルフアテンションとポイントワイズ、完全に接続されたレイヤーがあります。エンコーダとデコーダの両方に、位置エンコーディングが追加されます。位置エンコーディングは、入力埋め込みに加えられ、シーケンス内のトークンの位置を示します。位置エンコーディングには、サインとコサイン関数を使用することができます。また、マルチヘッドアテンションと呼ばれる機能を使用して、複数のアテンションレイヤーを並列に実行することができます。'),
 ('猫の柄についてまとめてください',
  "I'm sorry, but there is no information provided about the pattern of cats in the given context.")]

In [9]:
qa.clear()
qa.chat_history

[]

In [10]:
qa.generate('文書全体を要約し、日本語に翻訳して出力してください')

'この論文では、Transformerという新しいシーケンス変換モデルが提案されている。このモデルは、エンコーダ・デコーダアーキテクチャで最も一般的に使用される再帰層をマルチヘッドのセルフアテンションに置き換えることで、完全にアテンションに基づくモデルとなっている。このモデルは、従来の再帰層や畳み込み層に基づくアーキテクチャよりも高速に学習できるため、翻訳タスクにおいて従来のモデルよりも高い性能を発揮することができる。また、このモデルは、英語からドイツ語、英語からフランス語の翻訳タスクにおいて、従来のモデルよりも高い性能を発揮することができることが示されている。さらに、このモデルは、英語の構文解析タスクにおいても高い性能を発揮することが示されている。今後は、このモデルを他のタスクにも適用し、入力や出力のモダリティがテキスト以外の場合にも対応できるようにすることが目標とされている。\n\nこの論文では、新しいシーケンス変換モデルであるTransformerが提案されています。このモデルは、再帰層をマルチヘッドのセルフアテンションに置き換えることで、完全にアテンションに基づくモデルとなっています。このモデルは、従来のモデルよりも高速に学習できるため、翻訳タスクにおいて従来のモデルよりも高い性能を発揮することができます。また、このモデルは、英語からドイツ語、英語からフランス語の翻訳タスクにおいて、従来のモデルよりも高い性能を発揮することができることが示されています。さらに、このモデルは、英語の構文解析タスクにおいても高い性能を発揮することが示されています。今後は、このモデルを他のタスクにも適用し、入力や出力のモダリティがテキスト以外の場合にも対応できるようにすることが目標とされています。'